In [13]:
import os
import folium
import pandas as pd
import numpy as np
import networkx as nx

In [25]:
filedir= '/content/drive/MyDrive/Colab Notebooks/건축실무/도로 데이터'
year= 2021

nodes = pd.read_csv(os.path.join(filedir, str(year) +"_node.csv"))
links = pd.read_csv(os.path.join(filedir, str(year) +"_link.csv"))

In [26]:
nodes = nodes[['Id','NODE_NAME','latitude','longitude']]
links = links[['Source','Target']]

source_in = links['Source'].apply(lambda x : x in list(nodes['Id'])) # check Sources are in 분당
target_in = links['Target'].apply(lambda x : x in list(nodes['Id'])) # check Targets are in 분당
# source_in and target_in are boolean type pandas.Series which contains True or False
final_links = links[source_in & target_in] # contain if both target and source are contained in 분당

In [27]:
#그래프 생성

G = nx.Graph()
# R is the Earth's radius
R = 6371e3

for idx,row in nodes.iterrows():
    # add node to Graph G
    G.add_node(row['Id'],Label=row['NODE_NAME'],latitude=row['latitude'], longitude=row['longitude'])


for idx,row in final_links.iterrows():
    ## Calculate the distance between Source and Target Nodes
    '''
    lon1 = float(nodes[nodes['Id'] == row['Source']]['longitude'] * np.pi/180)
    lat1 = float(nodes[nodes['Id'] == row['Source']]['latitude'] * np.pi/180)
    lon2 = float(nodes[nodes['Id'] == row['Target']]['longitude'] * np.pi/180)
    lat2 = float(nodes[nodes['Id'] == row['Target']]['latitude'] * np.pi/180)
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1
    a = np.sin(d_lat/2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(d_lon/2) ** 2
    c = 2 * np.arctan2(a**0.5, (1-a) ** 0.5)
    d = R * c
    ''' 
    # Link attribute : 'Source', 'Target' and weight = 'Length between them'
    G.add_edge(row['Source'],row['Target'],weight = 1000)

#folium 시각화

# Positioning the Standard Point for our Folium Map
std_point = tuple(nodes.head(1)[['latitude','longitude']].iloc[0])
std_point

map_osm = folium.Map(location=std_point, zoom_start=13) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

for ix, row in nodes.iterrows():
    location = (row['latitude'], row['longitude']) # 위도, 경도 튜플
    folium.Circle(
        location=location,
        radius=G.degree[row['Id']] * 30, # 지름이 degree에 비례하도록 설정
        color='white',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        # popup=str(row['Id'])
    ).add_to(map_osm)
    # folium.Marker(location, popup=row['NODE_NAME']).add_to(map_osm)

kw = {'opacity': 0.5, 'weight': 2}
for ix, row in final_links.iterrows():
    start = tuple(nodes[nodes['Id']==row['Source']][['latitude','longitude']].iloc[0])
    end = tuple(nodes[nodes['Id']==row['Target']][['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        **kw,
    ).add_to(map_osm)
# it takes some time.....
map_osm

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/건축실무/도로 데이터/2014_node.csv')
df

,Unnamed: 0,Id,NODE_TYPE,NODE_NAME,TURN_P,REMARK,USERID,WORKSTATE,TMPID,IDCHECK,ORDERID,DEPT_CODE,STNL_REG,ID½Å±ÔºÎ¿©,latitude,longitude
0,0,2730011900,101,¼¼¹«¼­»ç°Å¸®,0.0,KOTI¿øº»¿¡¼­ ÃßÃâ(³»¾÷),korea,5,OEXT2730011900,NaN,NaN,NaN,273.0,NaN,37.129369,128.208248
1,1,2730019400,101,³ó°íÈÄ¹®,0.0,KOTI¿øº»¿¡¼­ ÃßÃâ(³»¾÷),korea,5,OEXT2730019400,NaN,NaN,NaN,273.0,NaN,37.136431,128.221262
2,2,2730013100,101,µ¿Çö±³Â÷·Î,0.0,KOTI¿øº»¿¡¼­ ÃßÃâ(³»¾÷),korea,5,OEXT2730013100,NaN,NaN,NaN,273.0,NaN,37.130525,128.220173
3,3,2730011600,101,¾àÃÊ½ÃÀåÀÔ±¸,0.0,KOTI¿øº»¿¡¼­ ÃßÃâ(³»¾÷),korea,5,OEXT2730011600,NaN,NaN,NaN,273.0,NaN,37.128414,128.219040
4,4,2730011500,101,¼¼¹«¼­»ç°Å¸®,0.0,KOTI¿øº»¿¡¼­ ÃßÃâ(³»¾÷),korea,5,OEXT2730011500,NaN,NaN,NaN,273.0,NaN,37.128120,128.211744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5068,5068,2720124500,108,ÃæÁÖºÐ±âÁ¡,1.0,NaN,NaN,5,TMPID03940010365,NaN,NaN,51110.0,272.0,NaN,37.038104,127.738326
5069,5069,2720124800,108,ÃæÁÖºÐ±âÁ¡,1.0,NaN,NaN,5,TMPID03940010367,NaN,NaN,51110.0,272.0,NaN,37.045241,127.734900
5070,5070,2720123600,108,¼­ÃæÁÖIC,1.0,NaN,NaN,5,TMPID03940010368,NaN,NaN,51110.0,272.0,NaN,37.008046,127.672700
5071,5071,2720123500,104,¼­ÃæÁÖ¿ä±Ý¼Ò,1.0,NaN,NaN,5,TMPID03940010369,NaN,NaN,51110.0,272.0,NaN,37.006839,127.673659


In [2]:
############################################################################################

In [28]:
filedir= '/content/drive/MyDrive/Colab Notebooks/건축실무/도로 데이터'
year= 2021

nodes = pd.read_csv(os.path.join(filedir, str(year) +"_node.csv"))
links = pd.read_csv(os.path.join(filedir, str(year) +"_link.csv"))

nodes = nodes[['Id','NODE_NAME','latitude','longitude']]
links = links[['Source','Target']]

source_in = links['Source'].apply(lambda x : x in list(nodes['Id'])) # check Sources are in 분당
target_in = links['Target'].apply(lambda x : x in list(nodes['Id'])) # check Targets are in 분당
# source_in and target_in are boolean type pandas.Series which contains True or False
final_links = links[source_in & target_in] # contain if both target and source are contained in 분당

df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/건축실무/도로 통계_예측 후.csv', delimiter= ',')
df['prediction']

nodes['prediction']= df['prediction']

In [29]:
#folium 시각화

# Positioning the Standard Point for our Folium Map
std_point = tuple(nodes.head(1)[['latitude','longitude']].iloc[0])
std_point

map_osm = folium.Map(location=std_point, zoom_start=13) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

for ix, row in nodes.iterrows():
    location = (row['latitude'], row['longitude']) # 위도, 경도 튜플
    folium.Circle(
        location=location,
        radius=row['prediction'] * 30, # 지름이 degree에 비례하도록 설정
        color='white',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        # popup=str(row['Id'])
    ).add_to(map_osm)
    # folium.Marker(location, popup=row['NODE_NAME']).add_to(map_osm)

kw = {'opacity': 0.5, 'weight': 2}
for ix, row in final_links.iterrows():
    start = tuple(nodes[nodes['Id']==row['Source']][['latitude','longitude']].iloc[0])
    end = tuple(nodes[nodes['Id']==row['Target']][['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        **kw,
    ).add_to(map_osm)
# it takes some time.....
map_osm

Output hidden; open in https://colab.research.google.com to view.